# Flappy Bird Junco Sim
2D flight sim for EECE5500 Mobile Robotics

In [239]:
############################### Import libraries ###############################
import os
import glob
import time
from datetime import datetime

import matplotlib.pyplot as plt
%matplotlib qt

import numpy as np
from numpy import matlib

import gym
# import control

In [240]:
class Drone():
    m = 3.2
    Iyy = 1/12*m*0.8**2

    S = 0.25
    c = 0.13

    Cl_0 = 0.5
    Cl_alpha = 0.17

    Cd_0 = 0.02
    K = 0.01

    Cm_0 = -0.05*0
    Cm_alpha = 0.4
    Cm_alpha_dot = -0.1*0
    Cm_delta_e = 0.5

    g = 9.81

    lidar_range = 40
    lidar_angle = 120*np.pi/180
    lidar_res = int(lidar_angle*180/np.pi)

    def __init__(self):
        self.x_thres = 1000
        self.z_thresHigh = 500
        self.z_thresLow = 0
        self.v_thres = 50
        self.theta_thres = np.pi/6
        self.theta_dot_thres = np.pi
        self.gamma_thres = np.pi/6
        self.alpha_dot_stale = 0

        self.limits = np.array([
            self.x_thres,
            self.z_thresHigh,
            self.z_thresLow,
            self.v_thres,
            self.theta_thres,
            self.theta_dot_thres,
            self.gamma_thres
        ])

        self.state = None
        self.prev_action = None
        self.steps_beyond_terminated = None

        self.time = 0.
        self.dt = 0.01
      
    def step(self, action):
        err_msg = f"{action!r} ({type(action)}) invalid"
        assert self.state is not None, "Call reset before using step method."
        
        self.time += self.dt

        #Extra states
        rho = np.random.normal(1.225,0.01)
        alpha = self.state[3] - self.state[4]
        self.q = 0.5*rho*self.state[2]**2
        
        #Control inputs
        thrust, delta_e = action

        # Forces
        Cl = self.Cl_0 + self.Cl_alpha*alpha
        Cd = self.Cd_0 + self.K*Cl**2
        Cm = self.Cm_0 + self.Cm_alpha*alpha + self.Cm_alpha_dot*self.alpha_dot_stale + self.Cm_delta_e*delta_e
        # print(self.Cm_0, self.Cm_alpha*alpha, self.Cm_alpha_dot*self.alpha_dot_stale, self.Cm_delta_e*delta_e)
        L = self.q*self.S*Cl
        D = self.q*self.S*Cd
        M = self.q*self.S*Cm
        x, z, v, theta, theta_dot, gamma = self.state

        #derivatives
        x_dot = v*np.cos(gamma)
        z_dot = v*np.sin(gamma)
        v_dot = (-D - self.m*self.g*np.sin(gamma) + thrust*np.cos(alpha)) / self.m
        #theta_dot = theta_dot
        theta_ddot = M / self.Iyy
        gamma_dot = (L - self.m*self.g*np.cos(gamma) -thrust*np.sin(alpha)) / (self.m*v)

        #No algebraic loops in my flight code 
        self.alpha_dot_stale = theta_dot - gamma_dot

        # integrate
        x += x_dot*self.dt
        z += z_dot*self.dt
        v += v_dot*self.dt
        theta += theta_dot*self.dt #Removed second order terms.  We don't do that here (Also needs to be 1/2dt^2 f"(t))
        theta_dot += theta_ddot * self.dt
        gamma += gamma_dot*self.dt

        state = np.array([x,z,v,theta,theta_dot,gamma])
        action = np.array([thrust,delta_e])

        self.state = (list(np.reshape(state,(6,))))

        terminated = bool(
            x < -self.x_thres
            or x > self.x_thres
            or z > self.z_thresHigh
            or z < self.z_thresLow
            or v < -self.v_thres
            or v > self.v_thres
            or theta < -self.theta_thres
            or theta > self.theta_thres
            or theta_dot < -self.theta_dot_thres
            or theta_dot > self.theta_dot_thres
            or gamma < -self.gamma_thres
            or gamma > self.gamma_thres
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            self.steps_beyond_terminated = 0
            reward = 0.0
        else:
            if self.steps_beyond_terminated == 0:
                print(self.state)
                gym.logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0.0

        return state, reward, terminated, False

    def reset(self):
        self.state = np.array([0,0,20,0,0,0])
        self.steps_beyond_terminated = None
        self.time = 0

        return self.state

    def close(self):
        pass

In [241]:
env = Drone()
state = env.reset()
action = np.array([1.5,0.000001])
fig, axs = plt.subplots(2, 2)
trajx = []
trajz = []
trajtheta = []
trajgamma = []
trajv = []

for t in range(1000):
    state, reward, done, _ = env.step(action)
    x = state[0]
    z = state[1]
    v = state[2]
    theta = state[3]
    theta_dot = state[4]
    gamma = state[5]
    trajx.append(x)
    trajz.append(z)
    trajtheta.append(theta)
    trajgamma.append(gamma)
    trajv.append(v)

axs[0, 0].plot(trajx, trajz)
axs[0, 0].set_ylabel("Height (m)")
axs[0, 1].plot(trajx, trajtheta)
axs[0, 1].set_ylabel("Theta (rad)")
axs[1, 0].plot(trajx, trajgamma)
axs[1, 0].set_ylabel("Gamma (rad)")
axs[1, 1].plot(trajx, trajv)
axs[1, 1].set_ylabel("Velocity (m/s)")

fig.show()

[0.6000116371523361, -7.404096977827849e-05, 20.001207334918327, 2.80607941446717e-08, 1.3766262509422575e-06, -0.000391126898634175]


/tmp/ipykernel_139757/2459268433.py:123: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  gym.logger.warn(


In [242]:
#################################### Testing ###################################

env = Drone()
state = env.reset()
action = np.array([1.5,1])

fig, ax = plt.subplots(1,figsize=(10,5))
traj = []

for t in range(1, 1000):
    
    state, reward, done, _ = env.step(action)
    x = state[0]
    z = -state[1]
    v = state[2]
    theta = -state[3]
    theta_dot = -state[4]
    gamma = state[5]
    traj.append((x,z))

    action = np.array([0,0])
        
    if v<25:
        action[0] = 200
    else:
        action[0] = 0
        
    l = 0.8
    plt.plot([x,x+0.25*l*np.cos(theta)],[z,z+0.25*l*np.sin(theta)],'k-')
    plt.plot([x,x-0.75*l*np.cos(theta)],[z,z-0.75*l*np.sin(theta)],'k-')
    plt.xlim(x-20,x+80)
    plt.ylim(z-25,z+25)

    beam = 50
    angle = 60*np.pi/180
    plt.plot([x,x+beam*np.cos(theta+angle/2)],[z,z+beam*np.sin(theta+angle/2)],'k--')
    plt.plot([x,x+beam*np.cos(theta-angle/2)],[z,z+beam*np.sin(theta-angle/2)],'k--')
    xs = x+beam*np.cos(theta+np.linspace(-angle/2,angle/2,10))
    zs = z+beam*np.sin(theta+np.linspace(-angle/2,angle/2,10))
    plt.plot(xs,zs,'k.',markersize=1)

    x,z = zip(*traj)
    plt.plot(x,z,'b-',markersize=1)
    
    plt.show()
    plt.pause(env.dt)

    
    if done:
        break
    else:
        ax.clear()
print(np.around(state,2), reward, done)
env.close()

[ 0.4  -0.   19.87  0.02 -0.8   0.03] 0.0 True


# Navigation Block

In [243]:
# plot results
def update_plot(ax,state,env,objects,beams=False,animate=False):

    # filter objects to ones in lidar cone
    def seenObjects(state,angle,beam,objects):

        x,y,theta = state

        # objects within range
        idx = np.logical_and((objects[0,:]>= x),(objects[0,:]<=x+beam))

        # above and below lines
        upper = np.tan(theta+angle/2)
        lower = np.tan(theta-angle/2)
        ind = np.arange(len(idx))
        for i in ind:
            if idx[i]:
                dx = objects[0,i] - x
                if y+upper*dx < objects[1,i] or y+lower*dx > objects[1,i]:
                    idx[i] = False

        return objects[:,idx]

    # intersection of line and circle
    def intersect(p1,p2,obj):
        x1 = p1[0] - obj[0]
        x2 = p2[0] - obj[0]
        y1 = p1[1] - obj[1]
        y2 = p2[1] - obj[1]

        r = obj[2]

        dx = x2-x1
        dy = y2-y1
        dr = np.sqrt(dx*dx + dy*dy)
        D = x1*y2 - x2*y1

        cross = False
        disc = r**2*dr**2 - D**2
        if disc>=0:
            cross = True
            xp = (D*dy + np.sign(dy)*dx*np.sqrt(r**2*dr**2 - D**2)) / dr**2
            xm = (D*dy - np.sign(dy)*dx*np.sqrt(r**2*dr**2 - D**2)) / dr**2
            yp = (-D*dx + np.abs(dy)*np.sqrt(r**2*dr**2 - D**2)) / dr**2
            ym = (-D*dx - np.abs(dy)*np.sqrt(r**2*dr**2 - D**2)) / dr**2
            
            dp = np.sqrt( (xp+obj[0])**2 + (yp+obj[1])**2 )
            dm = np.sqrt( (xm+obj[0])**2 + (ym+obj[1])**2 )
            if dp > dm:
                p2 = (xm+obj[0],ym+obj[1])
            else:
                p2 = (xp+obj[0],yp+obj[1])

        return p2, cross
    
    if animate:
        ax.clear()

    x,z,theta = state
    angle = env.lidar_angle
    beam = env.lidar_range
    num = env.lidar_res
    
    rays = np.zeros((2,num))
    if beams:
        objects = seenObjects(state,angle,beam,objects)
        
        angles = np.linspace(-angle/2,angle/2,num=num)
        for i,a in enumerate(angles):
            x1,y1 = x,z
            x2,y2 = x+beam*np.cos(theta+a),z+beam*np.sin(theta+a)
            
            # check for lidar intersection
            for obj in objects.T:
                p,cross = intersect((x1,y1),(x2,y2),obj)
                if cross:
                    if rays[0,i] == 0 or np.linalg.norm(rays[:,i]) > np.sqrt((p[0]-x1)**2+(p[1]-y1)**2):
                        x2,y2 = p
                        rays[0,i] = x2-x1
                        rays[1,i] = y2-y1

            if animate:
                ax.plot([x1,x2],[y1,y2],'k-')

    else:
        if animate:
            ax.plot([x,x+beam*np.cos(theta+angle/2)],[z,z+beam*np.sin(theta+angle/2)],'k--')
            ax.plot([x,x+beam*np.cos(theta-angle/2)],[z,z+beam*np.sin(theta-angle/2)],'k--')
            a = np.linspace(-angle/2,angle/2,int(num/3))
            xs = x+beam*np.cos(theta+a)
            zs = z+beam*np.sin(theta+a)
            ax.plot(xs,zs,'k.',markersize=1)

            xs = objects[0,:]
            ys = objects[1,:]
            s = objects[2,:]
            ax.scatter(xs,ys,50*s,'r')

    if animate:
        ax.set_ylabel('Height (m)')
        ax.set_xlim([x-0.5*beam,x+2.5*beam])
        ax.set_ylim([z-0.75*beam,z+0.75*beam])
        ax.set_aspect('equal')

    return rays

def update_grid(ax,beam,rays,res=1):
    
    r,c = int(1.5*beam/res), int(3*beam/res)
    grid = np.zeros((r,c))
    x,y = int(c/6),int(r/2)
    grid[y,x] = 1

    r,c = np.shape(rays)
    for i in range(c):
        if rays[0,i] != 0:
            grid[(y+rays[1,i]).astype(int),(x+rays[0,i]).astype(int)] = 1

    ax.clear()
    ax.imshow(grid,cmap='gray')
    ax.invert_yaxis()
    ax.set_aspect('equal')
    ax.set_yticklabels([])
    ax.set_xticklabels([])
    ax.set_xlabel('Distance (m)')
    ax.set_ylabel('Height (m)')

    return grid

In [244]:
animate = True
from PIL import Image
%matplotlib qt

if animate:
    plt.close()
    fig,(ax1,ax2,ax3) = plt.subplots(3,1,figsize=(6,10))

t = 50
num = 40
x_pos = np.linspace(50,280,num)
y_pos = np.random.uniform(low=-15,high=15,size=num)
sizes = np.random.uniform(low=0.5,high=2.0,size=num)
objects = np.stack((x_pos,y_pos,sizes))

for i in range(t):
    x = 2*i
    z = 5*np.cos(i/10)
    theta = 0.1*np.sin(i/5)
    state = [x,z,theta]

    update_plot(ax1,state,env,objects)
    ax1.set_title('Ground Truth')
    rays = update_plot(ax2,state,env,objects,beams=True)
    ax2.set_title('Lidar View')
    grid = update_grid(ax3,beam,rays)
    ax3.set_title('Occupancy Grid')

    plt.show()
    plt.pause(0.01)